In [8]:
import numpy as np
import cv2 as cv2
import matplotlib.pyplot as plt

In [9]:
import glob
birds = sorted(glob.glob("JPEGS/birds/*.jpg"))
bottle = sorted(glob.glob("JPEGS/bottle/*.jpg"))
cyclists = sorted(glob.glob("JPEGS/cyclists/*.jpg"))
surf = sorted(glob.glob("JPEGS/surf/*.jpg"))

# Optical flow Shi-Tomas

In [10]:
def find_and_save_lucas_kanade(frames,name):
    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                           qualityLevel = 0.6,
                           minDistance = 50,
                           blockSize = 5 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (30,30),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
                      flags=0)
    # Create some random colors
    color = np.random.randint(0,255,(100,3))

    # Take first frame and find corners in it
    old_frame = cv2.imread(frames[0],0)

    p0 = cv2.goodFeaturesToTrack(old_frame, mask = None, **feature_params)


    # Create a mask image for drawing purposes
    mask = np.zeros_like(cv2.cvtColor(old_frame,cv2.COLOR_GRAY2RGB))
    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(old_frame.shape[0])
    frame_height = int(old_frame.shape[1])

    #### Initialize writing video
    vid_name = "results/lucas_kanade_"+name+'.avi'
    out = cv2.VideoWriter(vid_name,cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width,frame_height))

    for frame in frames[1:]:
        new_frame = cv2.imread(frame,0)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, p0,                                                                              cv2.OPTFLOW_LK_GET_MIN_EIGENVALS, **lk_params)
        
        # # calculate optical flow
        # p1, st, err = cv2.calcOpticalFlowPyrLK(old_frame, new_frame, p0,                                                                              cv2.OPTFLOW_USE_INITIAL_FLOW, **lk_params)
        try:
            # Select good points
            good_new = p1[st==1]
            good_old = p0[st==1]

            # draw the tracks
            for i,(new,old) in enumerate(zip(good_new, good_old)):
                a,b = new.ravel()
                c,d = old.ravel()
                mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                new_frame = cv2.circle(new_frame,(a,b),5,color[i].tolist(),-1)
            img = cv2.add(cv.cvtColor(new_frame,cv2.COLOR_GRAY2RGB),mask)
            cv2.imshow('frame',img)
            
            #### Write video as file
            outframe = cv2.resize(img,(frame_width,frame_height))
            out.write(outframe)

            # Now update the previous frame and previous points
            old_frame = new_frame.copy()
            p0 = good_new.reshape(-1,1,2)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
        except:
            pass
    out.release()
    cv2.destroyAllWindows()

In [11]:
find_and_save_lucas_kanade(cyclists,"cyclists")
find_and_save_lucas_kanade(birds,"birds")
find_and_save_lucas_kanade(bottle,"bottle")
find_and_save_lucas_kanade(surf,"surf")

## Farneback Optical Flow

In [13]:
def find_and_save_Farneback(frames,name):
    old_frame = cv2.imread(frames[0],0)
    hsv = np.zeros_like(cv2.cvtColor(old_frame,cv2.COLOR_GRAY2RGB))
    hsv[..., 1] = 255
    
    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(old_frame.shape[0])
    frame_height = int(old_frame.shape[1])

    #### Initialize writing video
    vid_name = "results/Farneback_"+name+'.avi'
    out = cv2.VideoWriter(vid_name,cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width,frame_height))

    for frame in frames[1:]:
        new_frame = cv2.imread(frame,0)
        flow = cv2.calcOpticalFlowFarneback(
                    old_frame, new_frame, None, 0.5, 3, 15, 3, 5, 1.2, 2)

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang*180/np.pi/2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.imshow('frame2', bgr)

        outframe = cv2.resize(bgr,(frame_width,frame_height))
        out.write(outframe)

        old_frame = new_frame
        out.write(bgr)
        if cv2.waitKey(200) & 0xFF == ord('q'):
                break
    out.release()
    cv2.destroyAllWindows()

In [14]:
find_and_save_Farneback(cyclists,"cyclists")
find_and_save_Farneback(birds,"birds")
find_and_save_Farneback(bottle,"bottle")
find_and_save_Farneback(surf,"surf")

# Background Subtraction

## MOG

In [15]:
def backgounrd_subtractionMOG(frames,name):
    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG(nmixtures=5)

    frame = cv2.imread(frames[0],0)
    frame_width = int(frame.shape[0])
    frame_height = int(frame.shape[1])

    #### Initialize writing video
    vid_name = "results/MOG_"+name+'.avi'
    out = cv2.VideoWriter(vid_name,cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width,frame_height))

    for frame in frames:
        new_frame = cv2.imread(frame,0)
        fgmask = fgbg.apply(new_frame)
        new_frame[np.where(fgmask==0)]=0
        outframe = cv2.resize(new_frame,(frame_width,frame_height))
        outframe = cv2.cvtColor(outframe, cv2.COLOR_GRAY2BGR)
        out.write(outframe)
        cv2.imshow('frame', new_frame)
        if cv2.waitKey(200) & 0xFF == ord('q'):
                break
    out.release()
    cv2.destroyAllWindows()

In [16]:
backgounrd_subtractionMOG(cyclists,"cyclists")
backgounrd_subtractionMOG(birds,"birds")
backgounrd_subtractionMOG(bottle,"bottle")
backgounrd_subtractionMOG(surf,"surf")

## MOG2

In [17]:
def backgounrd_subtractionMOG2(frames,name):

    fgbg = cv2.createBackgroundSubtractorMOG2(
            history=100, varThreshold=25, detectShadows=False)

    frame = cv2.imread(frames[0],0)
    frame_width = int(frame.shape[0])
    frame_height = int(frame.shape[1])

    #### Initialize writing video
    vid_name = "results/MOG2_"+name+'.avi'
    out = cv2.VideoWriter(vid_name,cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width,frame_height))

    for frame in frames:
        new_frame = cv2.imread(frame,0)
        fgmask = fgbg.apply(new_frame)
        new_frame[np.where(fgmask==0)]=0
        outframe = cv2.resize(new_frame,(frame_width,frame_height))
        outframe = cv2.cvtColor(outframe, cv2.COLOR_GRAY2BGR)
        out.write(outframe)
        cv2.imshow('frame', new_frame)
        if cv2.waitKey(200) & 0xFF == ord('q'):
                break
    out.release()
    cv2.destroyAllWindows()

In [18]:
backgounrd_subtractionMOG2(cyclists,"cyclists")
backgounrd_subtractionMOG2(birds,"birds")
backgounrd_subtractionMOG2(bottle,"bottle")
backgounrd_subtractionMOG2(surf,"surf")

## GMG

In [19]:
def backgounrd_subtractionGMG(frames,name):

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    fgbg = cv2.bgsegm.createBackgroundSubtractorGMG(initializationFrames = 1,
                                                    decisionThreshold = 0.7)
    frame = cv2.imread(frames[0],0)
    frame_width = int(frame.shape[0])
    frame_height = int(frame.shape[1])

    #### Initialize writing video
    vid_name = "results/GMG_"+name+'.avi'
    out = cv2.VideoWriter(vid_name,cv2.VideoWriter_fourcc(*'mp4v'), 10, (frame_width,frame_height))

    for frame in frames:
        new_frame = cv2.imread(frame,0)
        fgmask = fgbg.apply(new_frame)
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

        new_frame[np.where(fgmask==0)]=0

        outframe = cv2.resize(new_frame,(frame_width,frame_height))
        outframe = cv2.cvtColor(outframe, cv2.COLOR_GRAY2BGR)
        out.write(outframe)
        cv2.imshow('frame', new_frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    out.release()
    cv2.destroyAllWindows()

In [20]:
backgounrd_subtractionGMG(cyclists,"cyclists")
backgounrd_subtractionGMG(birds,"birds")
backgounrd_subtractionGMG(bottle,"bottle")
backgounrd_subtractionGMG(surf,"surf")